<a href="https://colab.research.google.com/github/angelelliott/miso-bert/blob/main/facebook_misogyny_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers

In [ ]:
#!pip install emoji --upgrade

In [ ]:
# Import necessary libraries
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
import emoji
import re

In [ ]:
# load dataset

dataset  = pd.read_csv("/content/drive/MyDrive/myModels/MISOFB-22")

In [ ]:
dataset.head()

,Unnamed: 0,POST,MISOGYNOUS
0,1,Holaaaaa Ari??pues mira yo te recomendaría que...,0
1,3,El mío es dragón Ball Budokai Tenkaichi 3 Lati...,0
2,4,Rifate con una serie de Minecraft vanilla sin ...,0
3,7,El juego de tus ojos con tus labios y tu voz e...,0
4,8,Clahs royal me divierto verte jugar Clahs. Ere...,0


In [ ]:
dataset.dropna(inplace = True)

In [ ]:
len(dataset)

2468

In [ ]:
dataset = dataset.drop(columns=['Unnamed: 0'])

In [ ]:
# find length of the longest string
len(max(dataset['POST'].tolist(), key = len))

561

In [ ]:
#lowercase, change urls for url, usernames for usuario, change format of emojis

url = re.compile(r'http\S+')
mention = re.compile(r'@\S+')
dataset['POST'] =  dataset['POST'].str.lower()
dataset['POST'] = [url.sub('url', x) for x in dataset['POST']]
dataset['POST']= [mention.sub('usuario', x) for x in dataset['POST']]
dataset['POST'] = [emoji.demojize(x, language='es') for x in dataset['POST']]

In [ ]:
dataset.head()

,POST,MISOGYNOUS
0,holaaaaa ari??pues mira yo te recomendaría que...,0
1,el mío es dragón ball budokai tenkaichi 3 lati...,0
2,rifate con una serie de minecraft vanilla sin ...,0
3,el juego de tus ojos con tus labios y tu voz e...,0
4,clahs royal me divierto verte jugar clahs. ere...,0


In [ ]:
text = dataset['POST'].tolist()
label = dataset['MISOGYNOUS'].ravel()

In [ ]:
# split training set into train and test
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(text, label, test_size=.2)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.25)

In [ ]:
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/myModels/pretrained_model_concat3epc")
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/myModels/pretrained_model_concat3epc")

Some weights of the model checkpoint at /content/drive/MyDrive/myModels/pretrained_model_concat3epc were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My

In [ ]:
train_encodings = tokenizer(train_texts, return_tensors='pt', max_length=80, padding=True)
val_encodings = tokenizer(val_texts, return_tensors='pt', max_length=80, padding=True)
test_encodings = tokenizer(test_texts, return_tensors='pt', max_length=80, padding=True)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
class MisogynyDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = MisogynyDataset(train_encodings, train_labels)
val_dataset = MisogynyDataset(val_encodings, val_labels)
test_dataset = MisogynyDataset(test_encodings, test_labels)

In [ ]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=8, # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.0,               # strength of weight decay
    learning_rate = 5e-5,
    logging_dir='./logs',            # directory for storing logs
    optim = 'adamw_torch'
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1480
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 370
  Number of trainable parameters = 109852418
<ipython-input-17-7db7345b34a2>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=370, training_loss=0.3953409143396326, metrics={'train_runtime': 26.468, 'train_samples_per_second': 111.833, 'train_steps_per_second': 13.979, 'total_flos': 152111078880000.0, 'train_loss': 0.3953409143396326, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 494
  Batch size = 32
<ipython-input-17-7db7345b34a2>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.3666936755180359,
 'eval_runtime': 0.8366,
 'eval_samples_per_second': 590.479,
 'eval_steps_per_second': 19.125,
 'epoch': 2.0}

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
predictions = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 494
  Batch size = 32
<ipython-input-17-7db7345b34a2>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
compute_metrics(predictions)

{'accuracy': 0.8785425101214575,
 'f1': 0.8775510204081634,
 'precision': 0.8669354838709677,
 'recall': 0.8884297520661157}

In [ ]:
predictions.predictions.argmax(-1)

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,

In [ ]:
predictions.label_ids

array([0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,

In [ ]:
for i in range(len(test_texts)):
  if predictions.predictions.argmax(-1)[i] != predictions.label_ids[i]:
    print("Tweet:", test_texts[i])
    print("Correct label:", predictions.label_ids[i])
    print("Assigned label: ", predictions.predictions.argmax(-1)[i])


Tweet: ya te ví en página de adultos y no tienes nada de bueno
Correct label: 1
Assigned label:  0
Tweet: ¿y las patas n3gras? díganme el porqué.... por queeeeee!!!
Correct label: 0
Assigned label:  1
Tweet: mija los piropos no son acoso callejero
Correct label: 1
Assigned label:  0
Tweet: por lo que pude ver unas buenas mamadas si da 
Correct label: 1
Assigned label:  0
Tweet: quiero sostener tu mano para siempre 
Correct label: 0
Assigned label:  1
Tweet: marche pues
Correct label: 1
Assigned label:  0
Tweet: los secuestradores se la pasan bien rico
Correct label: 1
Assigned label:  0
Tweet: la primera es la hidro cuando recién te conoció la segunda es kim animandote xdd
Correct label: 0
Assigned label:  1
Tweet: mira el tamaño de esos magumbos
Correct label: 1
Assigned label:  0
Tweet: increíble pero es la pera verdad
Correct label: 0
Assigned label:  1
Tweet: a mí se me hace que el único acoso que ha recibido esta persona es que le dicen seguido que viste como pendeja
Correct label